# 5、初识神经网络（感知机和反向传播）

* 学习NN（Neural Network）的历史
* 层级感知机
* 理解反向传播
* “turn on” NN
* 用Keras实现基本的NN

反向传播（backpropagation），是NN的数学基础。

## 5.1 NN的要素

Rosenblatt的项目原本是“教”机器识别图像。

### 5.1.2 基本的感知机

![Basic](basic_perceptron.png)

这里的$x_i$表示一个feature，所有的特征构成特征向量$X$，每个特征有对应的权重（weight）$w_i$，加上一个偏差（bias），它们的和输入一个“激活函数（Activation Function）”，激活函数输出1或0。

偏差的存在是因为，神经元需要能够处理所有输入为0的情况，因为此时的加权和（即features和weights的点积）总是为0。

以这样的视角来查看感知机，它是非常简单的定义。

In [1]:
import numpy as np

inputs = [1, .2, .1, .05, .2]
weights = [.2, .12, .4, .6, .9]

input_vec = np.array(inputs)
weight_vec = np.array(weights)
bias_weight = .2

activatin_level = np.dot(input_vec, weight_vec) + (1 * bias_weight)
activatin_level

0.674

In [4]:
def act_func(level):
    threshold = 0.5
    return 1 if level > threshold else 0

perceptron_output = act_func(activatin_level)
perceptron_output

1

由以上运算过程可知，如果有了权重向量和激活函数，那么对于一个输入，就可以确定出输出是什么。但权重其实尚未确定出来，实际上它们才是“学习”的结果。

感知机学习时，每次根据预测结果与实际结果的差异对权重做细微的调整。它的起点一般是**随机值**，通常选自正态分布。有了足够多输入后，系统将能够“学习”到最佳参数。一个调整的示例如下：

In [5]:
# 期望结果不同，因此调整值
expected_output = 0

new_weights = []
for i, x in enumerate(inputs):
    new_weights.append(weights[i] + (expected_output - perceptron_output) * x)
weight_vec = np.array(new_weights)

weights, new_weights

([0.2, 0.12, 0.4, 0.6, 0.9],
 [-0.8, -0.08000000000000002, 0.30000000000000004, 0.5499999999999999, 0.7])

In [6]:
activatin_level = np.dot(input_vec, weight_vec) + (1 * bias_weight)
activatin_level

-0.41850000000000004

In [7]:
perceptron_output = act_func(activatin_level)
perceptron_output

0

经过调整，对于该输入，其输出正确了：）下面来解决经典的“OR”问题。

In [29]:
# inputs
inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
expected = [0, 1, 1, 1]
act_threshold = 0.5

from random import random

# init weights
weights = np.random.random(2) / 1000
print('weights', weights)

bias_weight = np.random.random() / 1000
print('bias weight', bias_weight)

weights [0.00049163 0.00084834]
bias weight 0.0004350974258090421


In [30]:
# 初始化接近原点的weights，开始训练
for epoch in range(5):
    correct_cnt = 0
    for idx, sample in enumerate(inputs):
        input_vec = np.array(sample)
        act_level = np.dot(input_vec, weights) + (bias_weight * 1)
        if act_level > act_threshold:
            output = 1
        else:
            output = 0
        if output == expected[idx]:
            correct_cnt += 1
        # 调整权重，weights与bias方式一致
        new_weights = []
        for i, x in enumerate(sample):
            new_weights.append(weights[i] + (expected[idx] - output) * x)
        bias_weight = bias_weight + (expected[idx] - output) * 1
        weights = np.array(new_weights)
    print(f'{correct_cnt} correct answers out of 4, for epoch {epoch}')

3 correct answers out of 4, for epoch 0
2 correct answers out of 4, for epoch 1
3 correct answers out of 4, for epoch 2
4 correct answers out of 4, for epoch 3
4 correct answers out of 4, for epoch 4


经过四轮学习，结果就全对了，后面再学习也不再有任何帮助，因为weights不会再有调整。这种情况称为**收敛（convergence）**。模型的误差函数如果有最小值，那么它可说是收敛的。现实情况下，不是每个函数都是收敛的。

基本感知机的固有问题是，它只能处理线性可分的数据。按上述计算过程来看，感知机只能做“线性回归”，故不能描述非线性关系。

单个感知机能力有限，但如果组合多个感知机，情况就变化了。（Rumelhardt McClelland）他们重新采用了一个古老的方法：反向传播。

对神经网络算法来说，即使它可以解决复杂的（非线性）问题，很长时间里，它也是过于高昂的算力，由此显得很不practical，在1990s到2010期间进入了第二次AI Winter。之后在算力、反向传播算法、大规模数据（猫、狗标注数据集之类）这些条件同时具备之后，NN再次返场。

### 5.1.3 损失函数（cost function / loss function）

通过定义损失函数，训练NN的目标即是：寻求使得损失函数具有最小值的参数。

**神经网络（Neural Network）**，是一个**神经元（Neuron）**的集合，其中的部分神经元之间建有连接。目前所用的是full connected network，即输入的每一个元素，会连接到下一层的每一个神经元。

### 5.1.4 激活函数（Activation Function）

目前使用的激活函数是阶跃函数，但反向传播的激活函数需要时非线性和连续可导的。常用的一种激活函数是sigmoid。

最简化的角度来看：输入所有训练数据，（总）误差会反向传播，以此更新每一个权重值，这样一个周期称为一个**epoch**。接下来可以重复执行多个epoch的工作。

中间还有一个**learning rate**的概念。

前面提到的最小值，是指对于所有输入得到总误差。

### 5.1.5 学习方法

输入所有数据，并以之调整权重的方法，称为*batch learning*，此方法可能只能找到局部最小值而非全局最小。有两种方法改进之：

* stochastic gradient descent
* mini-batch：更为常用

### 5.1.6 Keras：NN in Python

看下面XOR的例子：

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_train = np.array([[0], [1], [1], [0]])

model = Sequential()
n_neurons = 10

model.add(Dense(n_neurons, input_dim=2))  # input_dim只需要在第一层定义，后续会自动计算
model.add(Activation('tanh'))
model.add(Dense(1))  # output layer
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                30        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


解释：

* Dense: fully connected layers
* SDG: stochastic gd


In [50]:
sgd = SGD(lr=0.1)

# compile创建初识模型
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# no learning yet
model.predict(x_train)

array([[0.5       ],
       [0.6537022 ],
       [0.39829427],
       [0.5253802 ]], dtype=float32)

In [53]:
# NN不一定能够收敛，compile的结果有可能使得寻找全局最小值非常困难乃至不可能。如果发生这种情况，再次运行fit；或者重新compile后再fit。
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 657us/step - loss: 0.5762 - acc: 0.7500
Epoch 2/100
4/4 [==============================] - 0s 524us/step - loss: 0.5752 - acc: 0.7500
Epoch 3/100
4/4 [==============================] - 0s 529us/step - loss: 0.5742 - acc: 0.7500
Epoch 4/100
4/4 [==============================] - 0s 625us/step - loss: 0.5732 - acc: 0.7500
Epoch 5/100
4/4 [==============================] - 0s 575us/step - loss: 0.5721 - acc: 0.7500
Epoch 6/100
4/4 [==============================] - 0s 534us/step - loss: 0.5711 - acc: 0.7500
Epoch 7/100
4/4 [==============================] - 0s 817us/step - loss: 0.5701 - acc: 0.7500
Epoch 8/100
4/4 [==============================] - 0s 560us/step - loss: 0.5691 - acc: 0.7500
Epoch 9/100
4/4 [==============================] - 0s 570us/step - loss: 0.5680 - acc: 0.7500
Epoch 10/100
4/4 [==============================] - 0s 536us/step - loss: 0.5670 - acc: 0.7500
Epoch 11/100
4/4 [==============================] - 0s 491u

Epoch 88/100
4/4 [==============================] - 0s 583us/step - loss: 0.4659 - acc: 1.0000
Epoch 89/100
4/4 [==============================] - 0s 637us/step - loss: 0.4644 - acc: 1.0000
Epoch 90/100
4/4 [==============================] - 0s 655us/step - loss: 0.4628 - acc: 1.0000
Epoch 91/100
4/4 [==============================] - 0s 778us/step - loss: 0.4613 - acc: 1.0000
Epoch 92/100
4/4 [==============================] - 0s 583us/step - loss: 0.4598 - acc: 1.0000
Epoch 93/100
4/4 [==============================] - 0s 559us/step - loss: 0.4582 - acc: 1.0000
Epoch 94/100
4/4 [==============================] - 0s 545us/step - loss: 0.4567 - acc: 1.0000
Epoch 95/100
4/4 [==============================] - 0s 573us/step - loss: 0.4551 - acc: 1.0000
Epoch 96/100
4/4 [==============================] - 0s 603us/step - loss: 0.4536 - acc: 1.0000
Epoch 97/100
4/4 [==============================] - 0s 610us/step - loss: 0.4520 - acc: 1.0000
Epoch 98/100
4/4 [==============================] 

In [54]:
model.predict(x_train)

array([[0.24496274],
       [0.69815433],
       [0.59751886],
       [0.4661598 ]], dtype=float32)

In [55]:
model.predict_classes(x_train)

array([[0],
       [1],
       [1],
       [0]], dtype=int32)

In [56]:
# 保存模型
import h5py

model_struct = model.to_json()

with open('xor_model.json', 'w') as fout:
    fout.write(model_struct)
    
# trained weights
model.save_weights('xor_weights.h5')

### 5.1.8 more things

* 不同的激活函数（sigmoid，relu，hyperbolic tangent）
* 选择不同的learning rate
* 动态调整LR，使用momentum，寻找全局最小值
* 使用dropout
* 权重的regularization

### 5.1.9 归一化

normalization：将所有样本的所有特征值统一到某个特定的范围内。在NLP中，TF-IDF、one-hot、word2vec等已经属于normalized了，故不必太担心这一点。

# 6、理解词向量（Word2vec）

* 词向量是如何创建的
* 使用预先训练的词向量模型
* 使用词向量解决现实问题
* 可视化之
* 词嵌入的更多应用

NLP近些年的进展中，最令人兴奋的点之一是词向量的”发现“。前面的章节中，我们忽略了词的语境，即词周围的词。BOW将文档中的所有词统统放入了一个大袋子，这一章的技术则使用小得多的袋子，一般不超过10个token。

新介绍的技术能够识别出同义词、反义词，或同类词，如人、动物、地点等。而对于词、n-grams和文档的LSA，不能捕获词的意义，更不用说隐含的意思了。

词向量是词义的数值表示，包含字面及隐含语义，因此可以说词向量捕获了词的”内涵（connotation）“，这些含义合并为一个dense vector。

## 6.1 语义查询和类比（analogy）

有时候你想搜索的那个特定词忘记了，只能输入ta的相关信息如：She invented something to do with physics in Europe in the early 20th century，通过Google搜索结果，你会找到答案是”Marie Curie“。

而这种语义搜索可以通过词向量实现。我们要搜索的对象包含以下性质：女性、欧洲、物理学家、著名的，如果这些性质都有相应的向量，那么加起来得到”答案“的向量，然后可以去搜索最相似的那个。

Who is to nuclear physics what Louis Pasteur is to germs?

这是一个”之于“问题，即”谁之于核物理学，恰如巴斯德之于微生物学“？答案似乎应容易：

`answer_vector = wv['Louis_Pasteur'] - wv['germs'] + wv['physics']`

词向量善于解答此类问题。

## 6.2 词向量

2013年，Thomas Mikolov在Google发布了**Word2vec”。

Word2vec的强悍之处是，它是无监督的，只需要大量的数据集，而现在这个时代，在Google这样的公司，数据不缺。

Word2vec不需要标注，它预测的目标是一个词的临近词，这个正好在数据集之内。时间序列模型与NLP问题颇为相似，因为两者都是处理序列（词或数）。但预测本身不是Word2vec真正关心的，预测只是其手段，它关心的乃是词的内部表示。相比主题向量，词向量的表示捕获了更多的词义。

PS：通过repredict输入来学习的模型称为**autoencoder**。

Word2vec可能会学习到你意料不到的东西，比如每个词都会有一定的“geography”性质。语料库中的每个词都表示为向量，这类似于主题向量，惟词向量指向的语义更为具体和精确。

关于Word2vec的直观理解是，将向量理解为一个权重或分数的列表，每个权重值都关联到一个特定维度的语义。

Mikolov希望的是*vector-oriented reasoning*，简言之，可以对向量做加减运算。

主题向量构建自整篇文档，适用于文档分类、语义搜索和聚类，但对于短语和复合词来说，不够准确。

### 6.2.1 面向向量的推理

Word2vec的第一次公开露面是在2013年的ACL，据称其在回答类比问题时的精确度是LSA模型的四倍。

### 6.2.2 Word2vec的计算

一般有两种方式：

* skip-gram
* continuous bag-of-words

词向量的计算需要大量资源，不过也有公司提供了预训练的词向量如Google和Facebook。如果应用不是特定于某个领域的，那么可以使用预训练模型。

skip-gram中，通过一个词预测其临近的词，skip-gram是包含了gap的n-grams。它对应的NN是两层的，隐藏层包含n个神经元（n是词向量维度），输入层和输出层都包含M个神经元（M是词汇表大小）。激活函数是softmax。

softmax是通常用于分类问题的激活函数，softmax将输出限定在0到1之间，且其和为1，因此结果可视为“概率”。

以下为使用fasttext的预训练模型：

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/Users/andersc/data/word2vecs/wiki-news-300d-1M.vec')

# 寻找不相关词
model.doesnt_match('potato milk computer'.split())

# 组合几个词
model.most_similar(positive=['cooking', 'patatoes'], topn=5)

# init magic
model.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)

# [('queen', 0.7515910863876343),
#  ('monarch', 0.6741327047348022),
#  ('princess', 0.6713887453079224),
#  ('kings', 0.6698989868164062),
#  ('kingdom', 0.5971318483352661)]

### 6.2.4 训练自己的词向量表示

依然用到gensim。

* 预处理：句子集合；每个句子是token列表；

# 7、词就其位（CNN）

* NLP中使用NN
* 寻找词模式中的语义
* 创建一个CNN
* 将文本向量化以适用于NN
* 训练一个CNN
* 情感分析

语义不仅仅蕴含在词本身，也包含在词之间的空隙里，词的顺序以及组合。词之间的连接使得深度、信息和复杂性成为可能。对人来说，仅靠词或n-grams，也不能了解完整的语义，所谓“只言片语”。除了日常对话，伟大的作者通过特有的方式表达思想，这些方式也是“模式”。机器对这些模式一无所知，除非有人告诉它们。

NN发展极快，感知机之后是CNN和RNN之类。在Word2vec那儿可以看到，NN给NLP带来了全新的方法与视角。尽管NN的本初目的是让机器*学习*如何量化输入，但这个领域已经迅速从基本的分类、回归问题延伸至：翻译、chatbot、以某位作者的风格“写作”。

## 7.1 Learning meaning

一个词的本质和**它与其它词的关系**相关联，即看到一个句子或段落，它的整体意思，远超过通过字典把词一个个查出来所了解到的信息。关系至少有两种：

* Word order：词序改变，意思随之改变

The dog chased the cat.
The cat chased the dog.

* Word proximity（就近性？）：

The ship's hull, despite years at sea, millions of tons of cargo, and two mid-sea collisions, shone like new. (hull与shone的关系）

## 7.4 窄窗

CNN先是用于图像处理，但也可以通过词向量用于NLP上。

In [1]:
# handle padding input
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D

In [2]:
import glob
import os

from random import shuffle


def preprocess_data(filepath):
    pos_label = 1
    neg_label = 0
    dataset = []
    
    pos_path = os.path.join(filepath, 'pos')
    neg_path = os.path.join(filepath, 'neg')
    paths = [(pos_path, pos_label), (neg_path, neg_label)]
    for p, label in paths:
        for fn in glob.glob(os.path.join(p, '*.txt')):
            with open(fn, 'r') as f:
                dataset.append((label, f.read()))
                
    shuffle(dataset)
    
    return dataset

In [3]:
train_data = preprocess_data('data/aclImdb/train/')

In [4]:
train_data[0]

(1,
 "Xizao is a rare little movie. It is simple and undemanding, and at the same time so rewarding in emotion and joy. The story is simple, and the theme of old and new clashing is wonderfully introduced in the first scenes. This theme is the essence of the movie, but it would have fallen flat if it wasn't for the magnificent characters and the actors portraying them.<br /><br />The aging patriarch, Master Liu, is a relic of China's pre-expansion days. He runs a bath house in an old neighbourhood. Every single scene set in the bath house is a source of jelaousy for us stressed out, unhappy people. Not even hardened cynics can find any flaws in this wonderful setting.<br /><br />Master Liu's mentally handicapped son Er Ming is the second truly powerful character in the movie, coupled with his modern-life brother. The interactions between these three people, and the various visitors to the bath house, are amazingly detailed and heart-felt, with some scenes packing so much emotion it's b

In [9]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors

tokenizer = TreebankWordTokenizer()

In [7]:
word_vecs = KeyedVectors.load_word2vec_format('/Users/andersc/data/word2vecs/GoogleNews-vectors-negative300.bin.gz', 
                                              limit=200000,
                                              binary=True)

In [12]:
def tokenize_and_vectorize(dataset):
    vectorized = []
    
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vecs[token])
            except KeyError:
                pass
        vectorized.append(sample_vecs)
    return vectorized


def collect_expected(dataset):
    expected = [sample[0] for sample in dataset]
    return expected

In [13]:
vectorized_data = tokenize_and_vectorize(train_data)
expected = collect_expected(train_data)

In [27]:
split_point = int(len(vectorized_data) * 0.8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [15]:
max_len = 400 # maxlen of sentence
batch_size = 32 
embedding_dims = 300
filters = 250
kernel_size = 3 # like window size
hidden_dims = 250
epochs = 2


def pad_trunc(data, maxlen):
    new_data = []
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    
    for sample in data:
        if len(sample) >= maxlen:
            temp = sample[:maxlen]
        else:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        new_data.append(temp)
    return new_data

In [29]:
x_train = pad_trunc(x_train, max_len)
x_test = pad_trunc(x_test, max_len)

x_train = np.reshape(x_train, (len(x_train), max_len, embedding_dims))
y_trainstain = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), max_len, embedding_dims))
y_test = np.array(y_test)

In [30]:
print('Build model...')

model = Sequential()
model.add(Conv1D(
    filters,
    kernel_size,
    padding='valid',
    activation='relu',
    strides=1,
    input_shape=(max_len, embedding_dims)))

# max pooling;
model.add(GlobalMaxPooling1D())

# vanilla hidden layer;
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# project onto a single unit output layer, and squash it with a sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Training...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Build model...
Training...
Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 270s 14ms/step - loss: 0.3910 - acc: 0.8170 - val_loss: 0.3165 - val_acc: 0.8664
Epoch 2/2
20000/20000 [==============================] - 272s 14ms/step - loss: 0.2296 - acc: 0.9079 - val_loss: 0.3521 - val_acc: 0.8556


In [31]:
model_struct = model.to_json()
json_dump(model_struct, 'cnn_model.json')

model.save_weights('cnn_weights.h5')
print('Model saved.')

Model saved.


In [32]:
# from keras.models import model_from_json
# with open("cnn_model.json", "r") as json_file:
#     json_string = json_file.read()
# model = model_from_json(json_string)

# model.load_weights('cnn_weights.h5')

In [45]:
sample_1 = "I'm hate that the dismal weather that had me down for so long, when will it break! Ugh, when does happiness return?  The sun is blinding and the puffy clouds are too thin.  I can't wait for the weekend."
sample_2 = "I really love that place, the warm weather and amazing beach."
tests = [sample_1, sample_2]

sample_vec = tokenize_and_vectorize([(-1, sample) for sample in tests])
test_vec_list = pad_trunc(sample_vec, max_len)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), max_len, embedding_dims))
model.predict(test_vec)

array([[0.03859694],
       [0.99361515]], dtype=float32)

In [46]:
model.predict_classes(test_vec)

array([[0],
       [1]], dtype=int32)